In [1]:
!wget http://ufldl.stanford.edu/housenumbers/train_32x32.mat
!wget http://ufldl.stanford.edu/housenumbers/test_32x32.mat

--2024-02-18 23:43:09--  http://ufldl.stanford.edu/housenumbers/train_32x32.mat
Resolving ufldl.stanford.edu (ufldl.stanford.edu)... 171.64.68.10
Connecting to ufldl.stanford.edu (ufldl.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 182040794 (174M) [text/plain]
Saving to: ‘train_32x32.mat’

train_32x32.mat     100%[===================>] 173.61M  3.66MB/s    in 66s     

2024-02-18 23:44:16 (2.63 MB/s) - ‘train_32x32.mat’ saved [182040794/182040794]

--2024-02-18 23:44:16--  http://ufldl.stanford.edu/housenumbers/test_32x32.mat
Resolving ufldl.stanford.edu (ufldl.stanford.edu)... 171.64.68.10
Connecting to ufldl.stanford.edu (ufldl.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64275384 (61M) [text/plain]
Saving to: ‘test_32x32.mat’

test_32x32.mat      100%[===================>]  61.30M  7.61MB/s    in 18s     

2024-02-18 23:44:34 (3.36 MB/s) - ‘test_32x32.mat’ saved [64275384/64

In [12]:
import tensorflow as tf
from scipy.io import loadmat
import matplotlib.pyplot as plt
import numpy as np

In [4]:
trainset = loadmat("train_32x32.mat")
testset = loadmat("test_32x32.mat")

In [7]:
trainset['X'].shape, testset['X'].shape

((32, 32, 3, 73257), (32, 32, 3, 26032))

In [ ]:
idx = np.random.randint(0, trainset['X'].shape[3], size=36)
fig, axes = plt.subplots(6, 6, sharex=True, sharey=True, figsize=(5, 5))
for i, ax in zip(idx, axes.flatten()):
  ax.imshow(trainset['X'][:, :, :, i], aspect='equal')
  ax.xaxis.set_visible(False)
  ax.yaxis.set_visible(False)
plt.subplots_adjust(wspace=0, hspace=0)